# Data Loading
David Modjeska and Andrew Greene

This notebook isolates the code we use to load the data and do some last-minute pre-processing on it.

In [3]:
# Deal with our necessary importds

import datetime
import itertools as it
import numpy as np
import os.path as op
import pandas as pd
import scipy as sp
import sklearn.preprocessing as Preprocessing

from itertools import combinations
from scipy.io import mmread
from sklearn.decomposition import TruncatedSVD as tSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.display import display, HTML

In [4]:
### Specify processed data files to generate - full/partial, partial %, and train/test
### Note: this cell is the same in both notebooks

# load and clean full dataset?
#load_full = False
load_full = True  # AMG

# if not loading and cleaning full dataset, what sample percentage?
sample_percent = 10

if load_full:
    pct_str = ""
else: # not load_full
    pct_str = str(sample_percent) + "_pct"
    
# use training or testing data to generate minor files?
minor_use_train = True
if minor_use_train:
    mode_str = "train"
else: # not minor_use_train
    mode_str = "test"
    
### set intermediate file names
dir_str = "./intermediate_files/"

processed_data_train_file = dir_str + "processed_data_" + "train" + pct_str + ".json"
processed_data_test_file = dir_str + "processed_data_" + "test" + pct_str + ".json"

nlp_data_file = dir_str + "nlp_data_" + mode_str + pct_str + ".json"
nlp_data_train_file = dir_str + "nlp_data_" + mode_str + pct_str + ".json"
term_freqs_file = dir_str + "term_freqs_" + mode_str + pct_str + ".mtx"
diff_terms_file = dir_str + "diff_terms_" + mode_str + pct_str + ".json"

In [5]:
### load processed data
data = pd.read_json(processed_data_train_file)
data_nlp = pd.read_json(nlp_data_file)
desc_matrix_coo = mmread(term_freqs_file)
desc_matrix = sp.sparse.csr_matrix(desc_matrix_coo)
count_cols_df = pd.read_json(diff_terms_file)

count_cols_bool = count_cols_df.values > 0.0

In [6]:
print len(data) # Confirm that the number looks reasonable

173805


We want to do a little filtering here. As discussed in our writeup, we will limit ourselves to 36-month loans issued in the years 2011, 2012, and 2013

In [7]:
model_loan_term = 36
data_filtered = data[data.loan_term == model_loan_term]
data_filtered = data_filtered[pd.to_datetime(data_filtered.issue_date).dt.year.isin([2011,2012,2013])]
print len(data_filtered)

79052


In [8]:
# Make sure we have a reasonable distribution of issue dates
pd.to_datetime(data_filtered.issue_date).dt.year.value_counts()

2013    38957
2012    30170
2011     9925
Name: issue_date, dtype: int64

A quick top-level sanity-check of the data, using Pandas `describe` method (and transposing it so it first on screen better)

In [9]:
data_filtered.describe().T

/Users/davidmodjeska/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,count,mean,std,min,25%,50%,75%,max
annual_income,79052.0,6.903144e+04,5.626822e+04,5.000000e+03,4.200000e+04,6.000000e+04,8.400000e+04,7.141778e+06
cpi,79052.0,2.129961e-02,4.217134e-04,2.069586e-02,2.092454e-02,2.132448e-02,2.166709e-02,2.224093e-02
delinq_2_yrs,79052.0,1.832085e-01,4.819232e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
desc_len,38614.0,2.499745e+02,2.132272e+02,1.000000e+00,NaN,NaN,NaN,4.544000e+03
dti,79052.0,2.437067e-03,1.136559e-03,0.000000e+00,1.575150e-03,2.392426e-03,3.267855e-03,5.445891e-03
gdp,79052.0,9.997262e-01,3.283828e-05,9.995987e-01,9.997036e-01,9.997311e-01,9.997524e-01,9.997853e-01
id,79052.0,3.620877e+06,2.750039e+06,5.852400e+04,1.341303e+06,2.380480e+06,5.875565e+06,1.023482e+07
inquiry_6_mos,79052.0,7.952866e-01,9.543483e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
installment,79052.0,3.986631e+02,2.455526e+02,3.016000e+01,2.190400e+02,3.433900e+02,5.150800e+02,1.408130e+03
interest_rate,79052.0,1.283620e+01,3.946880e+00,5.420000e+00,9.990000e+00,1.299000e+01,1.561000e+01,2.589000e+01


We need to make a couple of adjustments to columns

In [10]:
# earliest_credit is not really a good indicator -- we want to know how long has elapsed since then
# See http://stackoverflow.com/questions/17414130/pandas-datetime-calculate-number-of-weeks-between-dates-in-two-columns
data_filtered['months_since_earliest_credit'] = (
    (pd.to_datetime(data_filtered.issue_date) - pd.to_datetime(data_filtered.earliest_credit))/np.timedelta64(1,'M')
).round()

In [11]:
# Separate the predictors (everything except "loan status") and the outcome
data_filtered_x = data_filtered.drop('loan_status', axis = 1)
data_filtered_y = data_filtered['loan_status']

In [12]:
# copy unstandardized columns for later profit calculation
profit_data = data_filtered_x[['installment', 'loan_amount', 'recoveries']]
recoveries_avg = profit_data.recoveries.sum() / float(np.count_nonzero(profit_data.recoveries))
data_filtered_x = data_filtered_x.drop('recoveries', axis = 1)

Standardize the data. (See the comment below for a detailed explanation of what that means)

In [13]:
# Certain columns in the raw data should not be in our model
columns_not_to_expand = [
    'description',     # free-text, so don't one-hot encode (NLP is separate)
    'verif_status',    # not sure why this is posing a problem....
    'loan_subgrade',   # tainted predictor
    'id',              # unique to each row
    'interest_rate',   # tainted predictor
    'index',           # unique to each row
    'issue_date',      # not useful in future, using economic indicators instead
    'earliest_credit', # has been converted to months_since_earliest_credit
]

In [14]:
# Given an input matrix X and the equivalent matrix X from the training set,
#
# (1) impute missing values (as "MISSING" for categorical, since the fact that 
# the value is missing may itself be significant; and using the median value
# for continuous predictors)
#
# (2) expand categorical predictors into a set of one-hot-encoded columns 
# (using 0 and 1, and limiting ourselves to the 50 most common values in the
# training set)
#
# (3) standardize continuous predictors using the mean and stdev of the
# training set

def expand_x(x, x_orig):
    x_expanded = pd.DataFrame()
    for colname in x_orig.columns:
        if colname in columns_not_to_expand:
            continue
        print colname, x_orig[colname].dtype
        if x_orig[colname].dtype == 'object':
            values = x[colname].fillna('MISSING')
            value_columns = x_orig[colname].fillna('MISSING').value_counts().index
            if len(value_columns) > 50:
                value_columns = value_columns[:50]
            for val in value_columns:
                x_expanded[colname + '__' + val.replace(' ', '_')] = (values == val).astype(int)
        else:
            values = x[colname].fillna(x[colname].median())
            sd = np.nanstd(x_orig[colname])
            if sd < 1e-10:
                sd = 1
            x_expanded[colname] = (values - np.nanmean(x_orig[colname]))/sd
    return x_expanded

### Split Data

In [15]:
# Get a more manageable sample
np.random.seed(1729)
sample_flags = np.random.random(len(data_filtered)) <= 0.25
print "Indexes computed" 

# train set
x_train = data_filtered_x.iloc[sample_flags, :]
x_expanded = expand_x(x_train, x_train)
print "Training set has %d rows" % (len(x_expanded),)

# test set
x_test_expanded = expand_x(data_filtered_x.iloc[~sample_flags, :], x_train)
print "Test set has %d rows" % (len(x_test_expanded),)

Indexes computed
address_state object
annual_income float64
cpi float64
delinq_2_yrs int64
desc_len float64
dti float64
employ_length object
employ_title object
gdp float64
home_owner object
initial_list_status object
inquiry_6_mos int64
installment float64
ipr float64
loan_amount int64
loan_purpose object
loan_term int64
months_since_last_record float64
open_accounts int64
revol_util float64
rir float64
total_accounts int64
unemploy float64
months_since_earliest_credit float64
Training set has 19719 rows
address_state object
annual_income float64
cpi float64
delinq_2_yrs int64
desc_len float64
dti float64
employ_length object
employ_title object
gdp float64
home_owner object
initial_list_status object
inquiry_6_mos int64
installment float64
ipr float64
loan_amount int64
loan_purpose object
loan_term int64
months_since_last_record float64
open_accounts int64
revol_util float64
rir float64
total_accounts int64
unemploy float64
months_since_earliest_credit float64
Test set has 59333 rows

In [16]:
# split response column
y = data_filtered_y.iloc[sample_flags]
y_test = data_filtered_y.iloc[~sample_flags]

In [17]:
# split profit data
profit_data_train = profit_data.iloc[sample_flags, :]
profit_data_test = profit_data.iloc[~sample_flags, :]

In [18]:
### filter NLP data
filter_flags = data_nlp.loan_term.values == model_loan_term
data_nlp_filtered = data_nlp.iloc[filter_flags]

x_nlp_filtered = data_nlp_filtered.drop('loan_status', 1)
y_nlp_filtered = data_nlp_filtered.loan_status

desc_matrix_filtered = desc_matrix[filter_flags]
count_cols_bool_filtered = count_cols_bool[filter_flags]

In [19]:
### split NLP data into training and testing sets
np.random.seed(1729)
train_flags = np.random.random(data_nlp_filtered.shape[0]) < 0.7

x_nlp_train = x_nlp_filtered.iloc[train_flags, :]
y_nlp_train = y_nlp_filtered.iloc[train_flags]

x_nlp_test = x_nlp_filtered.iloc[~train_flags, :]
y_nlp_test = y_nlp_filtered.iloc[~train_flags]

desc_matrix_train = pd.DataFrame(desc_matrix_filtered[train_flags, :].toarray())
desc_matrix_test = pd.DataFrame(desc_matrix_filtered[~train_flags, :].toarray())

count_cols_bool_train = pd.DataFrame(count_cols_bool_filtered[train_flags, :])
count_cols_bool_test = pd.DataFrame(count_cols_bool_filtered[~train_flags, :])

years_nlp = pd.to_datetime(x_nlp_train.issue_date).dt.year
years_nlp_test = pd.to_datetime(x_nlp_test.issue_date).dt.year

In [20]:
### match indexes

desc_matrix_train.index = x_nlp_train.index
desc_matrix_test.index = x_nlp_test.index

count_cols_bool_train.index = x_nlp_train.index
count_cols_bool_test.index = x_nlp_test.index

In [21]:
# inspect test proportion of good/bad loans
y_test.value_counts()

False    50334
True      8999
Name: loan_status, dtype: int64

In [22]:
# verify size of train set
np.count_nonzero(x_expanded.loan_amount)

19719

In [23]:
# be prepared to split stuff up by year of issue
years = pd.to_datetime(data_filtered_x.issue_date.iloc[sample_flags]).dt.year
years_test = pd.to_datetime(data_filtered_x.issue_date.iloc[~sample_flags]).dt.year

### Apply PCA to predictors

In [24]:
tsvd = tSVD(n_components = 100, random_state=1729)
tsvd.fit(x_expanded)
data_filtered_expanded_x_pca = pd.DataFrame(tsvd.transform(expand_x(data_filtered_x, x_train)))
data_filtered_expanded_x_pca.index = data_filtered_x.index
pca_cum_var_expl = np.cumsum(np.round(tsvd.explained_variance_ratio_, 4) * 100)

address_state object
annual_income float64
cpi float64
delinq_2_yrs int64
desc_len float64
dti float64
employ_length object
employ_title object
gdp float64
home_owner object
initial_list_status object
inquiry_6_mos int64
installment float64
ipr float64
loan_amount int64
loan_purpose object
loan_term int64
months_since_last_record float64
open_accounts int64
revol_util float64
rir float64
total_accounts int64
unemploy float64
months_since_earliest_credit float64


In [25]:
print "max variance explained", pca_cum_var_expl.max()
print "PCA: first and last columns where % variance explained >= 99:", \
            np.where(pca_cum_var_expl >= 99)[0][[0, -1]]

x_expanded_pca = data_filtered_expanded_x_pca.iloc[sample_flags, :73]
x_test_expanded_pca = data_filtered_expanded_x_pca.iloc[~sample_flags, :73]

max variance explained 99.62
PCA: first and last columns where % variance explained >= 99: [64 99]


In [26]:
tsvd = tSVD(n_components = 500, random_state=1729)
desc_matrix_filtered_pca = pd.DataFrame(tsvd.fit_transform(desc_matrix_filtered))
desc_matrix_filtered_pca.index = x_nlp_filtered.index
pca_cum_var_expl = np.cumsum(np.round(tsvd.explained_variance_ratio_, 4) * 100)

In [27]:
print "max variance explained", pca_cum_var_expl.max()
print "PCA: first and last columns where % variance explained >= 84:", \
            np.where(pca_cum_var_expl >= 84)[0][[0, -1]]

max variance explained 85.08
PCA: first and last columns where % variance explained >= 84: [221 499]


In [28]:
desc_matrix_pca = desc_matrix_filtered_pca.iloc[train_flags, :]
desc_matrix_test_pca = desc_matrix_filtered_pca.iloc[~train_flags, :]